In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

First we will import the libraries needed.

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense,Flatten, Reshape, LeakyReLU, Dropout, UpSampling2D
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Model
from tensorflow import keras

Now we will read the dataset using a tf.data.Dataset object.

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)
dataset = tf.data.Dataset.list_files('/kaggle/input/gan-getting-started/monet_jpg/*')
dataset

Now we will reads the file content as bytes, decodes it as a JPEG image using TensorFlow, and then converts the image to grayscale using the rgb_to_grayscale function.

In [ ]:
def load_image(x):
    byte_image = tf.io.read_file(x)
    img = tf.io.decode_jpeg(byte_image)
    img = tf.image.rgb_to_grayscale(img)
    return img

In [ ]:
dataset = dataset.map(load_image)

Visuallizing the data:

In [ ]:
image_gen = dataset.batch(4).as_numpy_iterator()
plot_img = image_gen.next()
fig, ax = plt.subplots(ncols = 4, figsize = (20,20))

for idx,imag in enumerate(plot_img):
    ax[idx].imshow(np.squeeze(imag))
plt.show()

In [ ]:
def scale_image(image):
 
    return image / 255

The next step is to perform data transformation and create batches of images for training the GANs

In [ ]:
dataset = dataset.map(scale_image)
dataset = dataset.cache()
dataset = dataset.shuffle(301)
dataset = dataset.batch(16)
dataset = dataset.prefetch(64)


In [ ]:
dataset.as_numpy_iterator().next().shape

Now we will build the generator

In [ ]:
def build_gen():
    seq_model = Sequential()
    
    # Input layer take in random values and reshape it into 16, 16, 128
    seq_model.add(Dense(16*16*128,input_dim = 16))
    seq_model.add(LeakyReLU(0.2))
    seq_model.add(Reshape((16,16,128)))
    
    
    # Upsampling layer 1 that will make it 32,32 128
    seq_model.add(UpSampling2D())
    seq_model.add(Conv2D(128,4,padding="same"))
    seq_model.add(LeakyReLU(0.2))
    
    # Upsampling layer 2
    seq_model.add(UpSampling2D())
    seq_model.add(Conv2D(128,4,padding="same"))
    seq_model.add(LeakyReLU(0.2))
    
    # Upsampling layer 3
    seq_model.add(UpSampling2D())
    seq_model.add(Conv2D(128,4,padding="same"))
    seq_model.add(LeakyReLU(0.2))
    
    # Upsampling layer 4
    seq_model.add(UpSampling2D())
    seq_model.add(Conv2D(128,4,padding="same"))
    seq_model.add(LeakyReLU(0.2))

    
    # Convolutional layer one 
    seq_model.add(Conv2D(128,4,padding="same"))
    seq_model.add(LeakyReLU(0.2))
    
    # Convolutional layer two for a better pattern detection
    seq_model.add(Conv2D(128,4,padding="same"))
    seq_model.add(LeakyReLU(0.2))
    
    
    # Convolutional layer to get to one channel
    seq_model.add(Conv2D(1,4,padding="same", activation="sigmoid"))
    return seq_model

In [ ]:
model = build_gen()
model.summary()

In [ ]:
image = model.predict(np.random.randn(2,16,1))
image

In [ ]:
fig, ax = plt.subplots(ncols = 2, figsize = (20,20))

for idx,im in enumerate(image):
    ax[idx].imshow(np.squeeze(im))

    

Now we will build the discrimenator

In [ ]:
def build_disc():
    model = Sequential()
    
    # Convenotional layer 1 for pattern detection
    model.add(Conv2D(32, 5, input_shape = (256,256,1)))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))
    
    # Convenotional layer 2
    model.add(Conv2D(64, 5))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))
    
    # Convenotional layer 3
    model.add(Conv2D(128, 5))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))
    
    #  Convenotional layer 4
    model.add(Conv2D(256, 5))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))

    
    # Flatten the output
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [ ]:
disc = build_disc()
disc.summary()

In [ ]:
plot_img.shape
image.shape

In [ ]:
## test the discriminator

disc.predict(image)

In [ ]:
gen_opt = Adam(learning_rate=0.0001)
disc_opt = Adam(learning_rate=0.00001)
gen_loss = BinaryCrossentropy()
disc_loss = BinaryCrossentropy()

Now we will train the model

In [ ]:
def train(real, gen_opt, disc_opt, gen_loss, disc_loss):
    gen = build_gen()
    disc = build_disc()
    
    
    for epoch in range(10):
        fake = gen(tf.random.normal((16, 16, 1)), training = False)
        image = real.as_numpy_iterator().next()
        with tf.GradientTape() as dtape:
            # Get the discriminator's predictions for real and fake images
            real_logits = disc(image, training = True)
            fake_logits = disc(fake, training = True)
            
            realfake_logits = tf.concat([real_logits, fake_logits], axis=0)
            
            # Create labels for real and fakes images
            logit_realfake = tf.concat([tf.zeros_like(real_logits), tf.ones_like(fake_logits)], axis=0)
            
            # Add some noise to the TRUE outputs to avoid overfitting
            noise_real = 0.15*tf.random.uniform(tf.shape(real_logits))
            noise_fake = -0.15*tf.random.uniform(tf.shape(fake_logits))
            logit_realfake += tf.concat([noise_real, noise_fake], axis=0)
            
            # Calculate loss  
            total_d_loss = disc_loss(logit_realfake, realfake_logits)
           
        # Update the generator's weights and biases
        dgradients = dtape.gradient(total_d_loss, disc.trainable_variables)
        disc_opt.apply_gradients(zip(dgradients, disc.trainable_variables))
            
        with tf.GradientTape() as gtape:

            fake = gen(tf.random.normal((16, 16, 1)), training = True)
            # Get the discriminator's predictions for fake images
            fake_logits = disc(fake, training = False)

            # Calculate the generator's loss
            g_loss = gen_loss(tf.zeros_like(fake_logits), fake_logits)

        # Update the generator's weights and biases
        gen_gradients = gtape.gradient(g_loss, gen.trainable_variables)
        gen_opt.apply_gradients(zip(gen_gradients, gen.trainable_variables))

In [ ]:
train(dataset,gen_opt, disc_opt, gen_loss, disc_loss)

In [ ]:
generator = build_gen()
imgs = generator.predict(tf.random.normal((7000, 16, 1)))
fig, ax = plt.subplots(ncols=4, nrows=4, figsize=(10,10))
for r in range(4): 
    for c in range(4): 
        ax[r][c].imshow(imgs[(r+1)*(c+1)-1])

In [ ]:
generator.save('generator.h5')
discriminator.save('discriminator.h5')

In [ ]:
import PIL
!mkdir ../images

In [ ]:
for i in range(7000):
    img = array_to_img(imgs[i], scale= True)
    image.save("../images/" + str(i) + ".jpg")

In [ ]:
import shutil
shutil.make_archive("/kaggle/working/images", 'zip', "/kaggle/images")